# Chaines de Markhov

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import numpy as np
from scipy import stats
import pysal


table = pd.read_csv('Databases/swaphist.csv', delimiter = ";")
table['Date'] = pd.to_datetime(table['Date'],dayfirst=True)
#table = scipy.delete(table,table.shape[0]-1,0)
print(min(table['1to5']))
np.array(table['1to5'].quantile([k/5 for k in range(0, 5)]))
output_3 = pd.read_csv('Databases/normal_implied_vol_300714.csv', delimiter = ";")
output_2 = pd.read_csv('Databases/normal_implied_vol_010807.csv', delimiter = ";")
#print(output_2)

11.2


# Définition des fonctions

### Calcul de la matrice de transition

In [2]:
#Calcule la place de l'élément dans un des n groupes
'''def place(elem,table,freq):
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/freq
    if table[elem]==xmax : freq-1
    else : return int((table[elem]-xmin)/delta)
    #return min(int((table[elem]-xmin)/delta),freq-1)'''

def place(elem, table, freq, quantiles):
    k = 0
    while k<freq and table[elem]>=quantiles[k]:
        k += 1
    return k-1

def placement(table, freq):
    quantiles = np.array(table.quantile([k/freq for k in range(0, freq)]))
    liste = []
    for elem in table :
        k=1
        while k<freq and elem>=quantiles[k]:
            k += 1
        liste.append(k-1)
    return np.array(liste)

#Calcule la matrice de passage (fréquence de passage d'un groupe à l'autre)
def matrice(table, freq):
    trans = np.zeros((freq,freq))
    n = table.size
    places = placement(table, freq)
    for i in range(0,n-1):
        trans[places[i],places[i+1]] += 1
    for i in range(0,freq):
        sum_ligne = np.sum(trans[i])
        trans[i] = trans[i]/sum_ligne
    return trans



#La matrice de transition est-elle bien stochastique ?
'''test = matrice(table['1to5'],20)
print(test)
for i in range(0,test.shape[0]):
    #print(sum(test[i]))
    if(sum(test[i])!=1.0): print(sum(test[i]))
'''

colors = ['RED','GREEN','BLUE','BLACK','ORANGE','PINK','BROWN','YELLOW','CYAN']
#fig, ax = plt.subplots()
#ax.plot_date(table['Date'], table['1to5'],'-',color='BLACK')
#ax.plot_date(table['Date'],20,'-',color='RED')
#fig.autofmt_xdate()

Les matrices de transitions ne sont pas stochastiques car certains intervalles de valeurs ne sont jamais visités : typiquement, les valeurs élevés atteintes durant le pic de 2008-2013. Les variations sont très importantes lors de cette période et les volatilités font des sauts importants d'une semaine à l'autre.

Il faudra prendre cela en compte lorsque l'on conduira les tests. En effet, les tests doivent être effectués sur des matrices stochastiques. On supprimera donc les colonnes et lignes vides.

### Fonctions auxiliaires pour le calcul des statistiques de test

In [3]:
#Une fonction pour supprimer les colonnes et lignes qui ne contiennent que des zeros
def zero(mat):
    liste = []
    for i in range(0,mat.shape[0]-1):
        if(sum(mat[i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,0)
    liste = []
    for i in range(0,mat.shape[1]-1): 
        if(sum(mat[:,i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,1)
    return mat

In [47]:
def A(table,i,freq,T):
    a=0
    delta = table.size/T
    tmp=0
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        #tmp =+ n(sous_table,i,freq)
        if n(sous_table,i,freq)!=0 : a+=1
    return a

def B(MT,i):
    B = []
    for j in range(0,MT.shape[1]):
        if MT[i,j]!=0 : B.append(j)
    return B

#elements de table appartenant à la i-ème classe
def n(sous_table,i,freq):
    tot = 0
    quantiles = np.array(sous_table.quantile([k/freq for k in range(0, freq)]))
    for elem in range(0,sous_table.size) :
        if place(elem,sous_table,freq,quantiles)==i : tot += 1
    return tot

def d(table,i,T,freq):
    delta = table.size/freq
    di = []
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        if n(sous_table,i,T)>0 : di.append(t)
    return di

def stat(table, MT, sous_table, sous_MT, T):
    tmp = 0
    for i in range(0,MT.shape[0]):
        #print("i : ",i)
        ni = n(sous_table,i, T)
        if ni>0 :
            Bi = B(MT,i)
            for j in Bi:
                tmp += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
                #if MT[i,j]!=0 : stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
    return tmp

def stat_tmp(table, MT, sous_table, sous_MT, T):
    dg_lbt = 0
    tmp = 0
    for i in range(0,MT.shape[0]):
        #print("i : ",i)
        ni = n(sous_table,i, T)
        if ni>0 :
            Bi = B(MT,i)
            for j in Bi:
                tmp += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
                dg_lbt = dg_lbt +1
                #if MT[i,j]!=0 : stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
    return tmp, dg_lbt

In [5]:
#Les matrices de transition de la table est-elle homogène ?
def homog(table, freq=50, T=5):
    for i,elem in enumerate(table):
        if i>0:
            Q,p = homog_temp(table[elem].values,freq,T)
            m1 = homog_int(table[elem].values,freq,T)
            m2 = homog_reg(table[elem].values,freq,T)
            print("p-valeur : ",p, " - résultat du test par intervalles : ",m1," - résultat du test par régions : ",m2)
            rep = p>0.05 and m1>0.5 and m2>0.5
            if rep : print("La série de valeur pour le swaption ",elem," est homogène")
            else : print("La série de valeur pour le swaption ",elem," n'est pas homogène")
    return 0

## Les matrices de transition sont elles bien homogènes ?

Pour que la matrice de transition calculée à partir des fréquences de transition ait un sens, il fait s'assurer qu'elle est bien homogène en temps. On va donc mener plusieurs test pour vérifier cette hypothèse lourde.

### Test d'homogénéité des intervalles par rapport à la matrice de transition globale

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On vérifie que, si on calcule la matrice de transition sur chacun des intervalles, on retrouve bien la matrice de transition calculée sur toutes les valeurs de la série par un test du khi-2.


Notre hypothèse nulle est donc : $H_0 = \{\forall t, \le T, \forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [49]:
def homog_temp(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    MT = matrice(table, freq)
    Q_tmp, dg_lbt_tmp = 0,0
    dg_lbt = 0
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q_tmp, dg_lbt_tmp = stat_tmp(table,MT,sous_table,sous_MT,T)
        Q += Q_tmp
        dg_lbt += dg_lbt_tmp
    
    dg_lib = 0
    for i in range(0, MT.shape[0]):
        ai = A(table,i,freq,T)
        if ai>0 : dg_lib += (len(B(MT,i))-1)*(ai-1)
    print("Degrés de liberté : ",dg_lib,dg_lbt)
    return Q, 1-stats.chi2.cdf(Q,dg_lbt)

### Test d'homogénéité par intervalle

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On va effectuer $T$ tests pour vérifier si chacune des $T$ matrices de transitions sont bien les mêmes que la matrice de transition globale.


Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t <T$ : $H_0^t = \{\forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [51]:
def homog_int(table, freq, T=3):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    MT = matrice(table, freq)
    for i in range(0,freq):
        Q = 0
        Di = d(table,i,T,freq)
        Bi = B(MT,i)
        for t in Di:
            ni = n(table,i,T)
            sous_table = table[int(t*delta): int((t+1)*delta)]
            sous_table.index = range(sous_table.shape[0])
            sous_MT = matrice(sous_table,freq)
            for j in Bi:
                Q += ni*(MT[i,j]-sous_MT[i,j])**2/MT[i,j]
        dg_lib = (len(Di)-1)*(len(Bi)-1)
        dg_lbt = len(Di)*len(Bi)
        p = 1-stats.chi2.cdf(Q,dg_lbt)

        print("Q : ",Q, " - dg de lbte : ",dg_lib,dg_lbt)
        print("p-valeur du test sur le ",t,"-ième intervalle : ",p)
        if p>0.05 : moy += 1
    
    print(moy," intervalles de temps sur ",T," sont homogènes")    
    return moy/T


    ''' for i in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q = stat(table,MT,sous_table,sous_MT,freq)
        dg_lib = (len(B(MT,i))-1)*n(table,i,freq)
        #for i in range(0, MT.shape[0]-1):
            #print(B(MT,i),A(table,i,freq,T))
            #dg_lib += (len(B(MT,i))-1)*(A(table,i,freq,T)-1)
        p = 1-stats.chi2.cdf(Q,dg_lib)'''

### Test région contre région

On subdivise toujours notre série de valeurs en un nombre $T$ d'intervalles. Pour finir, on vérifie si les matrices de transitions de deux périodes sont identiques.

Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t,u <T$ : $H_0^{t,u} = \{\forall i,j, p_{i,j}^t (t) = p_{i,j}^u\}$

In [52]:
def homog_reg(table, freq, T=3):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    t,u = 0,0
    for t in range(0,T):
        for u in range(t+1,T):
            sous_table1 = table[int(t*delta): int((t+1)*delta)]
            sous_table2 = table[int(u*delta): int((u+1)*delta)]
            sous_table1.index = range(int((t+1)*delta)-int(t*delta))
            sous_table2.index = range(int((u+1)*delta)-int(u*delta))
            sous_MT1 = matrice(sous_table1, freq)
            sous_MT2 = matrice(sous_table2, freq)
            Q = stat(sous_table1,sous_MT1,sous_table2,sous_MT2,T)
            Di = d(sous_table1,t,T,freq)
            Bi = B(sous_MT1,t)
            dg_lib = (len(Di)-1)*(len(Bi)-1)
            dg_lbt = len(Di)*len(Bi)
            MT = matrice(sous_table1, freq)
            #for i in range(0, MT.shape[0]-1):
                #dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
            p = 1-stats.chi2.cdf(Q,dg_lbt)
            print("Q : ",Q," - dg lib : ",dg_lib,dg_lbt)
            print("p-valeur du test entre le ",t+1,"-ième et le ",u+1,"-ième intervalle : ",p)
            if p>0.05 : moy += 1
    
    print(moy," paires d'intervalles de temps sur ",T," intervalles sont homogènes")    
    return 2*moy/(T*(T-1))

## Deux matrices de transition sont-elles égales ?

Maintenant, on peut tester si deux matrices de transition sont égales. On appliquera la statistique de test précédement définie aux deux matrices.

Notre hypothèse nulle est donc, si on note $(p_{i,j}^0)$ et $(p_{i,j}^1)$ les coefficents de chacune des deux matrices de transition : $H_0 = \{\forall i,j, p_{i,j}^0 = p_{i,j}^1\}$

# Tests sur toute la période 2001-2016

## Homogénéité

In [16]:
homog_temp(table['10to5'],5)

Degrés de liberté :  40 75


(360.63635912994732, 0.0)

In [55]:
homog_int(table['1to5'],5,T=5)

Q :  226.021582697  - dg de lbte :  4 10
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  383.195589  - dg de lbte :  8 15
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  125.273419088  - dg de lbte :  8 15
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  169.542440832  - dg de lbte :  8 15
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  87.2303959369  - dg de lbte :  4 10
p-valeur du test sur le  4 -ième intervalle :  1.89848137211e-14
0  intervalles de temps sur  5  sont homogènes


0.0

In [56]:
homog_reg(table['1to5'],5)

Q :  1.63042692212  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.950365047832
Q :  108.557390425  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.0
Q :  88.0644194037  - dg lib :  4 9
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  3.99680288865e-15
1  paires d'intervalles de temps sur  3  intervalles sont homogènes


0.3333333333333333

Comme on pouvait s'y attendre, les matrices de transition sur la période 2001-2016 ne sont homogène dans le temps pour aucun des swaptions. Nous allons donc restreindre notre études à de plus petits intervalles.

# Test sur la période 2001-2008

In [17]:
tab1 = table[0:292]

In [77]:
homog_reg(tab1['1to5'],5)
homog_reg(tab1['1to10'],5)
homog_reg(tab1['1to20'],5)
print('5')
homog_reg(tab1['5to5'],5)
homog_reg(tab1['5to10'],5)
homog_reg(tab1['5to20'],5)
print('10')
homog_reg(tab1['10to5'],5)
homog_reg(tab1['10to10'],5)
homog_reg(tab1['10to20'],5)

Q :  20.6781157057  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.00209552514999
Q :  1.49031121399  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.960135552723
Q :  32.8125843173  - dg lib :  6 12
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.00103537792631
1  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  26.3445390553  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.000192026746503
Q :  6.31218655514  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.389142450816
Q :  7.75412196427  - dg lib :  6 12
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.804037673732
2  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  18.6926603943  - dg lib :  4 9
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.0279367203365
Q :  21.71125  - dg lib :  4 9
p-valeur du test e

0.3333333333333333

In [60]:
homog_int(tab1['1to20'],10)

Q :  43.1072619048  - dg de lbte :  4 9
p-valeur du test sur le  2 -ième intervalle :  2.06005090742e-06
Q :  96.9565555556  - dg de lbte :  6 12
p-valeur du test sur le  2 -ième intervalle :  2.22044604925e-15
Q :  59.1674641148  - dg de lbte :  6 12
p-valeur du test sur le  2 -ième intervalle :  3.20008178845e-08
Q :  0  - dg de lbte :  -3 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -4 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3 0
p-valeur du test sur le  2 -ième intervalle :  nan
Q :  0  - dg de lbte :  -1 0
p-valeur du test sur le  2 -ième intervalle :  nan
0  intervalles de temps sur  3  sont homogènes


0.0

In [58]:
homog_temp(tab1['1to5'],3,T=5)

Degrés de liberté :  16 35


(53.657787191165319, 0.022692480993421804)

## Test sur la période 2008-2014

In [31]:
tab2 = table[292:658]
tab2.index = range(tab2.shape[0])

In [78]:
homog_reg(tab2['1to5'],5)
homog_reg(tab2['1to10'],5)
homog_reg(tab2['1to20'],5)
print('5')
homog_reg(tab2['5to5'],5)
homog_reg(tab2['5to10'],5)
homog_reg(tab2['5to20'],5)
print('10')
homog_reg(tab2['10to5'],5)
homog_reg(tab2['10to10'],5)
homog_reg(tab2['10to20'],5)

Q :  2.97833096973  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.811561573493
Q :  58.6792835014  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  8.34483593337e-11
Q :  34.5081082281  - dg lib :  6 12
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.000560635330098
1  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  4.79310185185  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.570610136761
Q :  80.0990740741  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  3.44169137634e-15
Q :  39.7350910364  - dg lib :  4 9
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  8.48582651225e-06
1  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  20.874857719  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.00193204105603
Q :  8.16928157139  - dg lib :  2 6
p-valeur 

0.0

In [35]:
homog_int(tab2['1to5'],5)

Q :  331.214836601  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  286.151631393  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  298.964916953  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  424.10857997  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  46.6988551736  - dg de lbte :  4
p-valeur du test sur le  4 -ième intervalle :  1.76181869094e-09
0  intervalles de temps sur  5  sont homogènes


0.0

In [21]:
homog_temp(tab2['1to5'],10,T=5)

Degrés de liberté :  96


(281.24660076254941, 0.0)

## Test sur la période 2014-2016

In [80]:
tab3 = table[658:]
tab3.index = range(tab3.shape[0])

In [81]:
homog_reg(tab3['1to5'],5)
homog_reg(tab3['1to10'],5)
homog_reg(tab3['1to20'],5)
print('5')
homog_reg(tab3['5to5'],5)
homog_reg(tab3['5to10'],5)
homog_reg(tab3['5to20'],5)
print('10')
homog_reg(tab3['10to5'],5)
homog_reg(tab3['10to10'],5)
homog_reg(tab3['10to20'],5)

Q :  6.0  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.423190081127
Q :  11.0694444444  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.0862537340013
Q :  4.84444444444  - dg lib :  4 9
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.847652069633
3  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  15.3666666667  - dg lib :  4 9
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.0813442253864
Q :  11.0407407407  - dg lib :  4 9
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.272926703962
Q :  5.91111111111  - dg lib :  4 9
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.74878068007
3  paires d'intervalles de temps sur  3  intervalles sont homogènes
Q :  6.3  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.390436440917
Q :  6.79444444444  - dg lib :  2 6
p-valeur du test entre le  1 -ième et le  

0.3333333333333333

In [33]:
homog_int(tab3['1to5'],20)

Q :  161.5  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  144.0  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  0.0
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  nan  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  nan  - dg de lbte :  8
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -3
p-valeur du test sur le  4 -ième intervalle :  nan
Q :  0  - dg de lbte :  -2
p-valeur du test sur le  4 -i

0.0

# Output

In [65]:
#print(table)
table['Date'] = pd.to_datetime(table['Date'],dayfirst=True)


#resultat= np.zeros((51,9))
swaptions = output_2.columns[2:]

In [66]:
#Calcule la moyenne des matrices de transition sur les 1000 itérations

def mat_output(output,elem,freq):
    matrice_freq = np.zeros((freq,freq))
    for i in range(1,1000):
        sous_table = output[output.Trial.values==i][elem]
        sous_table.index = range(sous_table.shape[0])
        matrice_freq += matrice(sous_table,freq)

    return matrice_freq/1000

In [71]:
def egales(table_in, table_out, elem, freq=5):
    MT1 = matrice(table_in[elem], freq)
    MT2 = mat_output(table_out, elem, freq)
    table_mean = table_out.groupby(table_out.Trial).mean()
    table_mean.index = range(table_mean.shape[0])
    Q, dg_lbt = stat_tmp(table_in[elem],MT1,table_mean[elem],MT2,freq)
    dg_lib = 0
    for i in range(0, MT1.shape[0]-1):
        dg_lib += (len(B(MT1,i))-1)
    print(dg_lib,dg_lbt)
    return Q, 1-stats.chi2.cdf(Q,dg_lbt)


In [82]:
#Sur toute la durée

for elem in table.columns[1:]:
    print(elem,egales(table,output_2,elem))

7 13
1to5 (858.87019232835894, 0.0)
7 13
1to10 (791.96667885981014, 0.0)
7 13
1to20 (610.87639310908446, 0.0)
7 13
5to5 (1208.8176163050896, 0.0)
7 13
5to10 (747.94538170748513, 0.0)
8 14
5to20 (903.87058418048525, 0.0)
9 15
10to5 (706.68832149527316, 0.0)
7 13
10to10 (907.37776034486092, 0.0)
7 13
10to20 (930.20531851609474, 0.0)
